In [2]:
from collections import defaultdict
from typing import Iterator, Tuple, Dict, List

# MapReduce для умножения матриц

In [4]:
from collections import defaultdict
from typing import Iterator, Tuple, Dict, List
import random

# Инициализируем генератор случайных чисел для воспроизводимости
random.seed(42)

# Создаем прямоугольные матрицы 90x100 и 100x90 со случайными числами от 0 до 100
first_matrix = [[random.randint(0, 100) for _ in range(100)] for _ in range(90)]
second_matrix = [[random.randint(0, 100) for _ in range(90)] for _ in range(100)]

# Результирующая матрица 90x90
result_matrix = [[0] * len(second_matrix[0]) for _ in range(len(first_matrix))]


def map_matrix_elements(first_matrix, second_matrix) -> Iterator[Tuple[Tuple[str, str], Tuple[str, int, int]]]:
    first_rows, first_cols = len(first_matrix), len(first_matrix[0])
    second_rows, second_cols = len(second_matrix), len(second_matrix[0])

    # Элементы из первой матрицы
    for row_idx in range(first_rows):
        for col_idx in range(first_cols):
            for result_col in range(second_cols):
                yield ((row_idx, result_col), ('first', col_idx, first_matrix[row_idx][col_idx]))

    # Элементы из второй матрицы
    for row_idx in range(second_rows):
        for col_idx in range(second_cols):
            for result_row in range(first_rows):
                yield ((result_row, col_idx), ('second', row_idx, second_matrix[row_idx][col_idx]))


def shuffle_matrix_elements(mapped_elements) -> Dict[Tuple[int, int], List[Tuple[str, int, int]]]:
    grouped_elements = defaultdict(list)
    
    for key, value in mapped_elements:
        grouped_elements[key].append(value)
    
    return grouped_elements


def reduce_matrix_elements(grouped_elements) -> None:
    for position, elements in grouped_elements.items():
        row_idx, col_idx = position
        first_matrix_elements = {}
        second_matrix_elements = {}

        # Разделяем элементы по исходным матрицам
        for matrix_type, index, value in elements:
            if matrix_type == 'first':
                first_matrix_elements[index] = value
            elif matrix_type == 'second':
                second_matrix_elements[index] = value

        # Вычисляем скалярное произведение
        cell_value = 0
        for index in first_matrix_elements:
            if index in second_matrix_elements:
                cell_value += first_matrix_elements[index] * second_matrix_elements[index]
        
        result_matrix[row_idx][col_idx] = cell_value


if __name__ == "__main__":
    print("Первая матрица (90x100) со случайными числами от 0 до 100:")
    print("Размер:", len(first_matrix), "x", len(first_matrix[0]))
    print("Полная матрица:")
    for i in range(len(first_matrix)):
        row = [f"{first_matrix[i][j]:3}" for j in range(len(first_matrix[0]))]
        print(f"Строка {i:2}: [{' '.join(row)}]")
    
    print("\n" + "="*100 + "\n")
    
    print("Вторая матрица (100x90) со случайными числами от 0 до 100:")
    print("Размер:", len(second_matrix), "x", len(second_matrix[0]))
    print("Полная матрица:")
    for i in range(len(second_matrix)):
        row = [f"{second_matrix[i][j]:3}" for j in range(len(second_matrix[0]))]
        print(f"Строка {i:2}: [{' '.join(row)}]")
    
    print("\n" + "="*100 + "\n")
    
    # Выполняем MapReduce pipeline для умножения матриц
    print("Выполняется умножение матриц через MapReduce...")
    mapped_elements = map_matrix_elements(first_matrix, second_matrix)
    shuffled_elements = shuffle_matrix_elements(mapped_elements)
    reduce_matrix_elements(shuffled_elements)
    
    print("\nРезультат умножения матриц (90x90):")
    print("Размер:", len(result_matrix), "x", len(result_matrix[0]))
    print("Полная результирующая матрица:")
    for i in range(len(result_matrix)):
        row = [f"{result_matrix[i][j]:6}" for j in range(len(result_matrix[0]))]
        print(f"Строка {i:2}: [{' '.join(row)}]")
    
    print("\n" + "="*100 + "\n")
    
    # Проверка корректности через прямое умножение (для первых нескольких элементов)
    print("Проверка корректности (первые 3x3 элемента):")
    for i in range(min(3, len(first_matrix))):
        for j in range(min(3, len(second_matrix[0]))):
            direct_calc = sum(first_matrix[i][k] * second_matrix[k][j] for k in range(len(second_matrix)))
            print(f"C[{i}][{j}] = {result_matrix[i][j]} (проверка: {direct_calc}) {'✓' if result_matrix[i][j] == direct_calc else '✗'}")
    
    # Статистика по результирующей матрице
    print("\nСтатистика результирующей матрицы:")
    all_values = [value for row in result_matrix for value in row]
    print(f"Минимальное значение: {min(all_values)}")
    print(f"Максимальное значение: {max(all_values)}")
    print(f"Среднее значение: {sum(all_values) / len(all_values):.2f}")

Первая матрица (90x100) со случайными числами от 0 до 100:
Размер: 90 x 100
Полная матрица:
Строка  0: [ 81  14   3  94  35  31  28  17  94  13  86  94  69  11  75  54   4   3  11  27  29  64  77   3  71  25  91  83  89  69  53  28  57  75  35   0  97  20  89  54  43  35  19  27  97  43  13  11  48  12  45  44  77  33   5  93  58  68  15  48  10  70  37  80  79  46  73  24  90   8   5  84  29  98  37  10  29  12  48  35  58  81  46  20  47  45  26  85  34  89  87  82   9  77  81  21  68  93  31  20]
Строка  1: [ 59  48  34  81  88  71  28  87  41  98  99   7  29   4  40  51  34   8  27  72  91  40  27  83  63  50  82  58  18  33  17  31  95  71  68  33  95  74  54  74  51  46  28  17  65  63  11  96   6  14  19  80  20  87  54  76   8  49  48  76  59  67  32  70   1  87  92  14  87  68  96  34  98  82  43  14  37  55  20  58   0  92  92  33  64  97  22  64  13  80  38  81  64  77  25  19  47  97  20  69]
Строка  2: [ 99  67   0  76  41  62   2  14  46  39  30   7  30  72  10  10  93  6

# MapReduce для линейеной регрессии

In [6]:

def map_regression(data_points: List[Tuple[float, float]]) -> Iterator[Tuple[str, float]]:
    for x_value, y_value in data_points:
        yield ("x_sum", x_value)
        yield ("y_sum", y_value)
        yield ("xy_sum", x_value * y_value)
        yield ("x_squared_sum", x_value * x_value)
        yield ("count", 1)


def shuffle_regression(mapped_pairs: Iterator[Tuple[str, float]]) -> Dict[str, List[float]]:
    grouped_results = defaultdict(list)
    
    for key, value in mapped_pairs:
        grouped_results[key].append(value)
    
    return dict(grouped_results)


def reduce_regression(grouped_stats: Dict[str, List[float]]) -> Tuple[float, float]:
    aggregated_stats = {}
    
    # Агрегируем значения по ключам
    for stat_key, values in grouped_stats.items():
        aggregated_stats[stat_key] = sum(values)
    
    # Извлекаем агрегированные статистики
    data_count = aggregated_stats["count"]
    x_total = aggregated_stats["x_sum"]
    y_total = aggregated_stats["y_sum"]
    xy_total = aggregated_stats["xy_sum"]
    x_squared_total = aggregated_stats["x_squared_sum"]
    
    # Проверяем достаточность данных
    if data_count < 2:
        raise ValueError("Для расчета регрессии нужно как минимум 2 точки данных")
    
    # Проверяем возможность вычисления
    calculation_denominator = data_count * x_squared_total - x_total ** 2
    if abs(calculation_denominator) < 1e-10:
        raise ValueError("Данные не позволяют вычислить коэффициенты регрессии")
    
    # Вычисляем коэффициенты регрессии
    slope = (data_count * xy_total - x_total * y_total) / calculation_denominator
    intercept = (y_total - slope * x_total) / data_count
    
    return slope, intercept


if __name__ == "__main__":
    # Исходные данные (x, y)
    sample_data = [
        (1.02, 2.7),
        (2.6, 1),
        (5, 4.5),
        (7.36, 2),
        (5.55, 9),
        (7.25, 6),
        (6.75, 3.44)
    ]

    try:
        # Выполняем MapReduce pipeline
        mapped_results = map_regression(sample_data)
        shuffle_results = shuffle_regression(mapped_results)
        regression_slope, regression_intercept = reduce_regression(shuffle_results)

        print(f"slope = {regression_slope:.4f}")
        print(f"intercept = {regression_intercept:.4f}")
        print(f"Регрессионное уравнение: y = {regression_slope:.4f}x + {regression_intercept:.4f}")

        # Пример предсказания
        test_x = 7
        predicted_y = regression_slope * test_x + regression_intercept
        print(f"Для x = {test_x}: предсказанное y = {predicted_y:.2f}")
        
    except ValueError as error:
        print(f"Ошибка вычислений: {error}")
    except Exception as error:
        print(f"Непредвиденная ошибка: {error}")

slope = 0.3902
intercept = 2.1107
Регрессионное уравнение: y = 0.3902x + 2.1107
Для x = 7: предсказанное y = 4.84
